# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:

# confirm that our new database was created
# List all the databases
database_names = mongo.list_database_names()

# Check if the "uk_food" database is in the list of databases
if "uk_food" in database_names:
    print("The 'uk_food' database exists.")
else:
    print("The 'uk_food' database does not exist.")

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
# List all the collections in the "uk_food" database
collection_names = db.list_collection_names()

# Print the collection names
print("Collections in the 'uk_food' database:")
for collection_name in collection_names:
    print(collection_name)


In [ ]:
# review the collections in our new database


In [ ]:
# review a document in the establishments collection
# Access the "establishments" collection
collection = db["establishments"]

# Find and display one document from the "establishments" collection
document = collection.find_one()

# Pretty print the document
pprint(document)


In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [ ]:
# Insert the new restaurant data into the collection
result = collection.insert_one(new_restaurant)


In [ ]:
# Check that the new restaurant was inserted
# Print the inserted document's ID
print("Inserted document ID:", result.inserted_id)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Define the query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Projection to return only the BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Find the document with the specified query and projection
result = collection.find_one(query, projection)

# Print the result
print(result)


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Define the query to find the "Penang Flavours" restaurant
query = {"BusinessName": "Penang Flavours"}

# Define the new BusinessTypeID
new_business_type_id = "12345"  # Replace with the correct BusinessTypeID

# Define the update query to set the new BusinessTypeID
update_query = {"$set": {"BusinessTypeID": new_business_type_id}}

# Update the document with the specified query
result = collection.update_one(query, update_query)


In [ ]:
# Confirm that the new restaurant was updated
# Print the number of documents modified
print("Number of documents modified:", result.modified_count)


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
# Define the query to count documents with LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the query
count = collection.count_documents(query)

# Print the count of documents
print("Number of documents with LocalAuthorityName as 'Dover':", count)


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
# Define the query to delete documents with LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete all documents that match the query
result = collection.delete_many(query)

# Print the number of documents deleted
print("Number of documents deleted:", result.deleted_count)


In [ ]:
# Check if any remaining documents include Dover
# Define the query to count documents with LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the query
count = collection.count_documents(query)

# Print the count of documents
print("Number of documents with LocalAuthorityName as 'Dover':", count)

In [ ]:
# Check that other documents remain with 'find_one'
# Find and display one document from the "establishments" collection
remaining_document = collection.find_one()

# Print the remaining document
print("Remaining document after deletion:")
pprint(remaining_document)


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
collection = db['establishments']

# Update longitude and latitude fields
for doc in collection.find():
    longitude = float(doc["geocode"]["longitude"]) if isinstance(doc.get("geocode", {}).get("longitude"), str) else doc["geocode"]["longitude"]
    latitude = float(doc["geocode"]["latitude"]) if isinstance(doc.get("geocode", {}).get("latitude"), str) else doc["geocode"]["latitude"]
    
    new_values = {
        "$set": {
            "geocode.longitude": longitude,
            "geocode.latitude": latitude
        }
    }
    collection.update_one({"_id": doc["_id"]}, new_values)

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
# Update the data type for RatingValue field in all documents
collection.update_many({}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])

# Verify the data type has been updated
cursor = collection.find({})
for document in cursor:
    print(document)

In [ ]:
# Check that the coordinates and rating value are now numbers
# Find and display one document from the "establishments" collection
sample_document = collection.find_one()

# Print the sample document to verify the data types
print("Sample document after data type changes:")
print(sample_document)
